## Importações e definições

In [1]:
import pandas as pd
from random import randint

inf = pd.read_csv('..\\Bases de Dados\\heroes_information.csv')
pwr = pd.read_csv('..\\Bases de Dados\\super_hero_powers.csv')

## Tratamento de dados: super_heroes_powers

### Tratamentos iniciais

In [2]:
display(pwr.head())

for col in pwr.columns:
    
    if pwr[col].dtype != bool:
        print(f'A coluna {col} não é do tipo booleana')

    if pwr[col].isna().sum() > 0:
        print(f'A coluna {col} possui {pwr[col].isna().sum()} dados faltantes.')    

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


A coluna hero_names não é do tipo booleana


In [3]:
# Convertendo valores booleanos para numericos
# Garante que os dados possam ser itnerpretados por qualquer algoritmo
# Permite realizar cálculos de estatística descritiva e de correlação entre atributos

pwr.replace({True: 1, False: 0}, inplace = True)
pwr.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A-Bomb,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abe Sapien,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abin Sur,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abomination,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Verificando se há algum personagem cadastrado duas ou mais vezes na base de dados
pwr.hero_names.value_counts()

hero_names
3-D Man            1
Namora             1
Naruto Uzumaki     1
Nathan Petrelli    1
Nebula             1
                  ..
Evilhawk           1
Exodus             1
Fabian Cortez      1
Falcon             1
Zoom               1
Name: count, Length: 667, dtype: int64

### Agrupamento de superpoderes

In [5]:
# Vamos agrupar poderes de acordo com semelhanças entre eles, para poder reduzir a dimensionalidade dos dados
superpowers_dict = {
    'Physical Skills': ['Accelerated Healing', 'Stamina', 'Toxin and Disease Resistance', 'Self-Sustenance', 'Regeneration', 'Cold Resistance', 'Super Breath', 'Immortality', 'Fire Resistance', 'Heat Resistance', 'Radiation Immunity', 'Agility', 'Super Strength', 'Durability', 'Reflexes', 'Enhanced Touch', 'Longevity', 'Peak Human Condition', 'Stealth', 'Natural Armor', 'Adaptation', 'Camouflage', 'Dexterity'],
    'Energy Manipulation': ['Sonic Scream', 'Matter Absorption', 'Probability Manipulation', 'Audio Control', 'Melting', 'Illumination',  'Heat Generation', 'Light Control', 'Force Fields', 'Hyperkinesis', 'Telekinesis', 'Power Nullifier', 'Molecular Dissipation', 'Omnipotent', 'Molecular Manipulation', 'Molecular Combustion', 'Power Absorption', 'Power Augmentation',  'Radiation Absorption', 'Radiation Control', 'Nova Force', 'Power Cosmic', 'Energy Resistance', 'Energy Manipulation', 'Energy Constructs', 'Energy Beams', 'Energy Blasts', 'Energy Armor', 'Energy Absorption'],
    'Elemental Powers': ['Magnetism', 'Toxin and Disease Control', 'Electrical Transport', 'Animal Control', 'Plant Control', 'Weather Control', 'Elemental Transmogrification', 'Seismic Power', 'Electrokinesis', 'Element Control', 'Terrakinesis', 'Wind Control', 'Water Control', 'Cryokinesis', 'Fire Control'],
    'Mental and Psychic Powers': ['Intelligence', 'Mind Blast', 'Enhanced Memory', 'Illusions', 'Thirstokinesis', 'Omnilingualism', 'Insanity', 'Precognition', 'Psionic Powers', 'Empathy', 'Hypnokinesis', 'Mind Control Resistance', 'Mind Control', 'Clairvoyance', 'Telepathy Resistance', 'Telepathy'],
    'Space-Time Manipulation': ['Projection', 'Gravity Control', 'Anti-Gravity', 'Astral Travel', 'Astral Projection', 'Portal Creation', 'Dimensional Awareness', 'Dimensional Travel', 'Time Manipulation', 'Time Travel'],
    'Body Transformation': ['Hair Manipulation', 'Invisibility', 'Invulnerability', 'Symbiote Costume', 'Vitakinesis', 'Substance Secretion', 'Duplication', 'Biokinesis', 'Elasticity', 'Intangibility', 'Cloaking', 'Density Control', 'Size Changing', 'Shapeshifting'],
    'Supernatural Forces': ['Grim Reaping', 'Banish', 'Death Touch', 'Possession', 'Phoenix Force', 'Odin Force', 'Animation', 'Reality Warping', 'The Force', 'Resurrection', 'Summoning', 'Darkforce Manipulation', 'Magic Resistance', 'Magic'],
    'Tech Skills': ['Changing Armor', 'Omnitrix', 'Power Suit', 'Qwardian Power Ring', 'Lantern Power Ring', 'Technopath/Cyberpath'],
    'Perception Skills': ['Intuitive aptitude', 'Danger Sense', 'Power Sense', 'Intuitive aptitude', 'Omnipresent', 'Omniscient', 'Photographic Reflexes', 'Enhanced Smell', 'Vision - Cryo', 'Vision - Thermal', 'Vision - Heat', 'Vision - X-Ray', 'Vision - Infrared', 'Vision - Night', 'Vision - Microscopic', 'Vision - Telescopic', 'Radar Sense', 'Sonar', 'Echolocation', 'Spatial Awareness', 'Enhanced Senses', 'Enhanced Hearing', 'Enhanced Sight'],
    'Locomotion Skills': ['Gliding', 'Sub-Mariner', 'Underwater breathing', 'Web Creation', 'Wallcrawling', 'Super Speed', 'Speed Force', 'Jump', 'Levitation', 'Phasing', 'Teleportation', 'Flight'],
    'Combat Skills': ['Animal Attributes', 'Natural Weapons', 'Weapon-based Powers', 'Animal Oriented Powers', 'Weapons Master', 'Marksmanship'],
}

cod_pwr = pd.DataFrame({}, columns = ['hero_names'] + list(superpowers_dict.keys()))

for i in pwr.index:

    cod_pwr = pd.concat([cod_pwr,
                        pd.DataFrame({
                            'hero_names':                [pwr.hero_names.loc[i]],
                            'Physical Skills':           [pwr[superpowers_dict['Physical Skills']].loc[i].sum()],
                            'Energy Manipulation':       [pwr[superpowers_dict['Energy Manipulation']].loc[i].sum()],
                            'Elemental Powers':          [pwr[superpowers_dict['Elemental Powers']].loc[i].sum()],
                            'Mental and Psychic Powers': [pwr[superpowers_dict['Mental and Psychic Powers']].loc[i].sum()],
                            'Space-Time Manipulation':   [pwr[superpowers_dict['Space-Time Manipulation']].loc[i].sum()],
                            'Body Transformation':       [pwr[superpowers_dict['Body Transformation']].loc[i].sum()],
                            'Supernatural Forces':       [pwr[superpowers_dict['Supernatural Forces']].loc[i].sum()],
                            'Tech Skills':               [pwr[superpowers_dict['Tech Skills']].loc[i].sum()],
                            'Perception Skills':         [pwr[superpowers_dict['Perception Skills']].loc[i].sum()],
                            'Locomotion Skills':         [pwr[superpowers_dict['Locomotion Skills']].loc[i].sum()],
                            'Combat Skills':             [pwr[superpowers_dict['Combat Skills']].loc[i].sum()]
                        })], 
                        ignore_index = True)  
    

cod_pwr

,hero_names,Physical Skills,Energy Manipulation,Elemental Powers,Mental and Psychic Powers,Space-Time Manipulation,Body Transformation,Supernatural Forces,Tech Skills,Perception Skills,Locomotion Skills,Combat Skills
0,3-D Man,3,0,0,0,0,0,0,0,0,1,0
1,A-Bomb,7,0,0,0,0,0,0,0,0,0,0
2,Abe Sapien,9,0,0,2,0,0,0,0,1,2,2
3,Abin Sur,0,0,0,0,0,0,0,1,0,0,0
4,Abomination,4,0,0,1,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
662,Yellowjacket II,0,1,0,0,0,1,0,0,0,1,0
663,Ymir,5,0,1,0,0,0,0,0,0,0,0
664,Yoda,5,3,0,4,0,1,1,0,1,2,2
665,Zatanna,0,1,5,1,0,0,1,0,0,0,0


## Tratamento de dados: heroes_information

In [6]:
display(inf.head())

# Desconsiderando a primeira coluna (índice)
inf.drop(columns = 'Unnamed: 0', inplace = True)

# Desconsiderando linhas duplicadas
inf.drop_duplicates(inplace = True)

display(inf.info())
display(inf[['Height', 'Weight']].describe())

,Unnamed: 0,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


<class 'pandas.core.frame.DataFrame'>
Index: 733 entries, 0 to 733
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        733 non-null    object 
 1   Gender      733 non-null    object 
 2   Eye color   733 non-null    object 
 3   Race        733 non-null    object 
 4   Hair color  733 non-null    object 
 5   Height      733 non-null    float64
 6   Publisher   718 non-null    object 
 7   Skin color  733 non-null    object 
 8   Alignment   733 non-null    object 
 9   Weight      731 non-null    float64
dtypes: float64(2), object(8)
memory usage: 63.0+ KB


None

,Height,Weight
count,733.000000,731.000000
mean,102.528649,44.050616
std,139.521460,130.806346
min,-99.000000,-99.000000
25%,-99.000000,-99.000000
50%,175.000000,62.000000
75%,185.000000,90.000000
max,975.000000,900.000000


### Tratamentos iniciais

In [7]:
# Tratamento de dados nulos
# Esta etapa é necessária pois a maioria dos modelos de ML não é capaz de lidar diretamente com dados nulos

# Para a Editora, vamos considerar '-', pois é um dado categórico
inf['Publisher'].fillna('-', inplace = True)

# Para o peso, vamos utilizar nosso conhecimento de negócio para entender os dados 
display(inf[inf.Weight.isna()])

inf.loc[286, 'Weight'] = 198416000 # (valor canônico para o Godzilla de "Godzilla: King of the Monsters" de 2019)
inf.loc[389, 'Weight'] = 316800    # (valor aproximado para "Kong: Skull Island" de 2017)

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
286,Godzilla,-,-,Kaiju,-,108.0,-,grey,bad,NaN
389,King Kong,Male,yellow,Animal,Black,30.5,-,-,good,NaN


In [8]:
# Verificando se há algum personagem cadastrado mais de uma vez
name_count = inf.name.value_counts()
name_count = name_count[name_count > 1]

for name in name_count.index:
    display(inf[inf.name == name])

# Muitos personagens possuem mais de uma versão, então mantive os 3 homens aranhas, por exemplo
# Também, muitos personagens tem a versão feminina e masculina
# Namor: uma das amostras possui muitos dados faltantes, alem de estar com peso e altura negativos. Irei desconsiderar essa amostra
# Blue Beetle, 	Batgirl, Blizzard, 	Atom, Goliath: análogo ao Namor

inf.drop(index = [50, 62, 115, 123, 289, 481], inplace = True)

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
622,Spider-Man,Male,hazel,Human,Brown,178.0,Marvel Comics,-,good,74.0
623,Spider-Man,-,red,Human,Brown,178.0,Marvel Comics,-,good,77.0
624,Spider-Man,Male,brown,Human,Black,157.0,Marvel Comics,-,good,56.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
695,Vindicator,Female,green,Human,Red,165.0,Marvel Comics,-,good,54.0
696,Vindicator,Male,-,-,-,-99.0,Marvel Comics,-,good,-99.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
496,Nova,Male,brown,Human,Brown,185.0,Marvel Comics,-,good,86.0
497,Nova,Female,white,Human / Cosmic,Red,163.0,Marvel Comics,gold,good,59.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
617,Speedy,Male,-,Human,-,-99.0,DC Comics,-,good,-99.0
618,Speedy,Female,green,Human,Brown,-99.0,DC Comics,-,good,-99.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
68,Batman,Male,blue,Human,black,188.0,DC Comics,-,good,95.0
69,Batman,Male,blue,Human,Black,178.0,DC Comics,-,good,77.0


...

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
155,Captain Marvel,Female,blue,Human-Kree,Blond,180.0,Marvel Comics,-,good,74.0
156,Captain Marvel,Male,blue,Human,Black,193.0,DC Comics,-,good,101.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
96,Black Canary,Female,blue,Human,Blond,165.0,DC Comics,-,good,58.0
97,Black Canary,Female,blue,Metahuman,Blond,170.0,DC Comics,-,good,59.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
49,Atom,Male,blue,-,Red,178.0,DC Comics,-,good,68.0
50,Atom,Male,-,-,-,-99.0,DC Comics,-,good,-99.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
289,Goliath,Male,-,-,-,-99.0,Marvel Comics,-,good,-99.0
290,Goliath,Male,-,Human,-,-99.0,Marvel Comics,-,good,-99.0


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
47,Atlas,Male,brown,Mutant,Red,183.0,Marvel Comics,-,good,101.0
48,Atlas,Male,blue,God / Eternal,Brown,198.0,DC Comics,-,bad,126.0


### Tratamento de outliers numericos

In [9]:
# Existem personagens com pesos muito desproporcionais a suas alturas. Não entraremos nesse mérito nesta análise
inf[['name', 'Height', 'Weight']].sort_values(by = 'Height', ascending = False)

,name,Height,Weight
255,Fin Fang Foom,975.0,18.0
272,Galactus,876.0,16.0
302,Groot,701.0,4.0
463,MODOK,366.0,338.0
718,Wolfsbane,366.0,473.0
...,...,...,...
494,Nite Owl II,-99.0,-99.0
493,Nina Theroux,-99.0,-99.0
492,Niki Sanders,-99.0,-99.0
177,Claire Bennet,-99.0,-99.0


In [10]:
# Porém, também há personagens cadastrados com peso e altura negativa (todos tendo exatamente -99.0 cm ou -99.0 pounds, o que me leva a crer que tenha havido um erro sistemico na medição dos dados).
# Então vamos verificar se isso faz mesmo sentido

display(inf[['Height', 'Weight']].describe())
display(inf[(inf.Weight < 0) | (inf.Height < 0)])

,Height,Weight
count,727.000000,7.270000e+02
mean,104.191884,2.734052e+05
std,138.883744,7.358834e+06
min,-99.000000,-9.900000e+01
25%,-99.000000,-9.900000e+01
50%,175.000000,6.300000e+01
75%,185.000000,9.100000e+01
max,975.000000,1.984160e+08


,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0
6,Adam Monroe,Male,blue,-,Blond,-99.0,NBC - Heroes,-,good,-99.0
14,Alex Mercer,Male,-,Human,-,-99.0,Wildstorm,-,bad,-99.0
15,Alex Woolsly,Male,-,-,-,-99.0,NBC - Heroes,-,good,-99.0
18,Allan Quatermain,Male,-,-,-,-99.0,Wildstorm,-,good,-99.0
...,...,...,...,...,...,...,...,...,...,...
712,Weapon XI,Male,-,-,-,-99.0,Marvel Comics,-,bad,-99.0
713,White Canary,Female,brown,Human,Black,-99.0,DC Comics,-,bad,-99.0
715,Wildfire,Male,-,-,-,-99.0,DC Comics,-,good,-99.0
723,Wondra,Female,-,-,-,-99.0,Marvel Comics,-,good,-99.0


In [11]:
# Vamos inicialmente estimar valores de peso e altura para os personagens com proporções humanas

# Raças com proporções aproximadas
rpa = ['Human / Clone', 'Human-Vulcan', 'Human-Spartoi', 'Zombie', 'Clone', 'Mutant / Clone', 
       'Vampire', 'Metahuman', 'Amazon', 'Human-Kree', 'Human / Altered', 'Atlantean', 'Asgardian', 'Kryptonian', 'Human']

# Peso: 
print('Peso')
display(inf[(inf.Weight > 0) & (inf.Race.isin(rpa)) & (inf.Gender == 'Male')].Weight.describe()) # Para homens: média = 111 | std = 91
display(inf[(inf.Weight > 0) & (inf.Race.isin(rpa)) & (inf.Gender == 'Female')].Weight.describe()) # Para mulheres: média = 80 | std = 77

for i in inf.index:

    if (inf.Weight.loc[i] < 0) & (inf.Race.loc[i] in rpa):

        if inf.Gender.loc[i] == 'Female':
            inf.loc[i, 'Weight'] = 80 + (77 / 2) * (randint(-100, 100) / 100) # Dividi op std por 2 pra o personagem não ficar com peso muito baixo ou negativo

        elif inf.Gender.loc[i] == 'Male':
            inf.loc[i, 'Weight'] = 111 + (91 / 2) * (randint(-100, 100) / 100)

# Altura:
print('Altura')
display(inf[(inf.Height > 0) & (inf.Race.isin(rpa)) & (inf.Gender == 'Male')].Height.describe()) # Para homens: média = 183 | std = 21
display(inf[(inf.Height > 0) & (inf.Race.isin(rpa)) & (inf.Gender == 'Female')].Height.describe()) # Para mulheres: média = 175 | std = 21

for i in inf.index:

    if (inf.Height.loc[i] < 0) & (inf.Race.loc[i] in rpa):

        if inf.Gender.loc[i] == 'Female':
            inf.loc[i, 'Height'] = 175 + 21 * (randint(-100, 100) / 100)

        elif inf.Gender.loc[i] == 'Male':
            inf.loc[i, 'Height'] = 183 + 21 * (randint(-100, 100) / 100)

#  OBSERVAÇÃO

# Para os personagens das demais raças, poderíamos fazer um processo análogo ao feito para os aproximadametne humanos. Mas como o intuito do desafio não é de fato treinar um modelo que tenha o melhor 
# desempenho possível (e o tempo está um pouco escasso), e sim avaliar meu raciocícinio ao desenvolver um projeto de ML, irei desconsiderar todos os dados de personagens com altura ou peso iguais a -99.

inf = inf[(inf.Height != -99) & (inf.Weight != -99)]

Peso


count    140.000000
mean     111.528571
std       91.097737
min       14.000000
25%       79.000000
50%       88.000000
75%      104.000000
max      855.000000
Name: Weight, dtype: float64

count     49.000000
mean      80.102041
std       77.505442
min       41.000000
25%       54.000000
50%       58.000000
75%       63.000000
max      495.000000
Name: Weight, dtype: float64

Altura


count    148.000000
mean     183.466216
std       21.785102
min       64.000000
25%      178.000000
50%      183.000000
75%      188.000000
max      287.000000
Name: Height, dtype: float64

count     51.000000
mean     175.098039
std       21.427324
min      137.000000
25%      165.000000
50%      170.000000
75%      175.000000
max      297.000000
Name: Height, dtype: float64

### Agrupamento de variáveis categoricas

In [12]:
# Primeiramente, vamos agrupar os possíveis valores algumas caracteristicas dos personagens

# Para a editora
def publisher_group_by(publisher_name):

    if (publisher_name != 'Marvel Comics') & (publisher_name != 'DC Comics'):
        return 'Others'
    else:
        return publisher_name

inf['Publisher_group_by'] = inf['Publisher'].apply(publisher_group_by)
print('Editora')
display(inf['Publisher_group_by'].value_counts())

# Para a raça
def race_group_by(race_name):

    race_dict = {'Human and Variation': ['Human', 'Human / Radiation', 'Human / Clone', 'Human-Kree', 'Human / Altered', 'Human-Vuldarian', 'Human / Cosmic', 'Human-Vulcan', 'Human-Spartoi'],
                'Alien': ['Symbiote', 'Kryptonian', 'Ungaran', 'Xenomorph XX121', 'Zen-Whoberian', 'Bolovaxian', 'Strontian', 'Kakarantharaian', 'Martian', 'Korugaran', 'Luphomoid', 'Tamaranean', 'Yautja', 'Talokite',  "Yoda's species"],
                'Genetically Altered Beings': ['Mutant', 'Inhuman', 'Metahuman', 'Mutant / Clone'], 
                'Divine and Semidivine': ['Cosmic Entity', 'God / Eternal', 'New God', 'Asgardian', 'Demi-God', 'Eternal'],
                'Underwater': ['Atlantean', 'Icthyo Sapien'],
                'Mythical': ['Amazon', 'Demon', 'Vampire', 'Zombie', 'Frost Giant'],
                'Artificial Beings': ['Cyborg', 'Android', 'Bizarro', 'Clone'],
                'Others': ['Kaiju', 'Saiyan', 'Gorilla', 'Flora Colossus', 'Animal']}
    
    for group in race_dict.keys():        
        if race_name in race_dict[group]:
            return group
        
    return 'Not Identified'

inf['Race_group_by'] = inf['Race'].apply(race_group_by)
print('Raça')
display(inf['Race_group_by'].value_counts())

# Para a cor dos olhos
def eye_color_group_by(eye_color):

    if (eye_color != 'blue') & (eye_color != 'brown') & (eye_color != 'green') & (eye_color != 'red'):
        return 'Others'
    else:
        return eye_color

inf['Eye_Color_group_by'] = inf['Eye color'].apply(eye_color_group_by)
print('Cor dos olhos')
display(inf['Eye_Color_group_by'].value_counts())

# Para a cor dos cabelos
def hair_color_group_by(hair_color):

    if (hair_color != 'Black') & (hair_color != 'Blonde') & (hair_color != 'No Hair') & (hair_color != 'Red') & (hair_color != 'Brown'):
        return 'Others'
    else:
        return hair_color

inf['Hair_Color_group_by'] = inf['Hair color'].apply(hair_color_group_by)
print('Cor dos cabelos')
display(inf['Hair_Color_group_by'].value_counts())

# Como a esmagadora maioria (487 / 583) dos valores de "Skin color" são '-', vou desconsiderar essa variável
inf.drop(columns = 'Skin color', inplace = True)

# Apenas para simplificar, vamos desconsiderar os 3 dados onde Alignment == '-'
inf = inf[inf.Alignment != '-']

Editora


Publisher_group_by
Marvel Comics    325
DC Comics        166
Others            52
Name: count, dtype: int64

Raça


Race_group_by
Human and Variation           228
Not Identified                175
Genetically Altered Beings     58
Alien                          27
Artificial Beings              15
Divine and Semidivine          15
Mythical                       12
Others                          7
Underwater                      6
Name: count, dtype: int64

Cor dos olhos


Eye_Color_group_by
blue      205
brown     118
Others    111
green      69
red        40
Name: count, dtype: int64

Cor dos cabelos


Hair_Color_group_by
Others     217
Black      145
Brown       79
No Hair     56
Red         46
Name: count, dtype: int64

### Codificando as variáveis categóricas (codificação one-hot)

In [13]:
cod_inf = inf[['name']]

col = ['Gender', 'Race_group_by', 'Eye_Color_group_by', 'Hair_Color_group_by']
cod_inf = pd.concat([cod_inf, pd.get_dummies(inf[col], columns = col)], axis = 1)

cod_inf = pd.concat([cod_inf, inf[['Height', 'Weight']]], axis = 1)

col = ['Publisher_group_by', 'Alignment']
cod_inf = pd.concat([cod_inf, pd.get_dummies(inf[col], columns = col)], axis = 1)

# Para conseguir reduzir ainda mais a dimensionalidade dos dados, vou desconsiderar as colunas com classificações "Others" (Por exemplo: Se o personagem não é
# da DC nem da Marvel, clarametne ele se enquadra em Others). O mesmo vale para alinhamento neutro.

for col in cod_inf.columns:
    if 'Others' in col:
        cod_inf.drop(columns = col, inplace = True)

cod_inf.drop(columns = 'Gender_-', inplace = True)
cod_inf.drop(columns = 'Alignment_neutral', inplace = True)

cod_inf = pd.concat([cod_inf, inf[['Gender', 'Publisher_group_by', 'Race_group_by', 'Alignment']]], axis = 1)

cod_inf.head()

,name,Gender_Female,Gender_Male,Race_group_by_Alien,Race_group_by_Artificial Beings,Race_group_by_Divine and Semidivine,Race_group_by_Genetically Altered Beings,Race_group_by_Human and Variation,Race_group_by_Mythical,Race_group_by_Not Identified,...,Height,Weight,Publisher_group_by_DC Comics,Publisher_group_by_Marvel Comics,Alignment_bad,Alignment_good,Gender,Publisher_group_by,Race_group_by,Alignment
0,A-Bomb,False,True,False,False,False,False,True,False,False,...,203.0,441.0,False,True,False,True,Male,Marvel Comics,Human and Variation,good
1,Abe Sapien,False,True,False,False,False,False,False,False,False,...,191.0,65.0,False,False,False,True,Male,Others,Underwater,good
2,Abin Sur,False,True,True,False,False,False,False,False,False,...,185.0,90.0,True,False,False,True,Male,DC Comics,Alien,good
3,Abomination,False,True,False,False,False,False,True,False,False,...,203.0,441.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad
5,Absorbing Man,False,True,False,False,False,False,True,False,False,...,193.0,122.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad


## Unificando as duas bases

In [14]:
cod_inf.columns

Index(['name', 'Gender_Female', 'Gender_Male', 'Race_group_by_Alien',
       'Race_group_by_Artificial Beings',
       'Race_group_by_Divine and Semidivine',
       'Race_group_by_Genetically Altered Beings',
       'Race_group_by_Human and Variation', 'Race_group_by_Mythical',
       'Race_group_by_Not Identified', 'Race_group_by_Underwater',
       'Eye_Color_group_by_blue', 'Eye_Color_group_by_brown',
       'Eye_Color_group_by_green', 'Eye_Color_group_by_red',
       'Hair_Color_group_by_Black', 'Hair_Color_group_by_Brown',
       'Hair_Color_group_by_No Hair', 'Hair_Color_group_by_Red', 'Height',
       'Weight', 'Publisher_group_by_DC Comics',
       'Publisher_group_by_Marvel Comics', 'Alignment_bad', 'Alignment_good',
       'Gender', 'Publisher_group_by', 'Race_group_by', 'Alignment'],
      dtype='object')

In [15]:
print(inf.shape, cod_inf.shape)
cod_inf.head()

(540, 13) (540, 29)


,name,Gender_Female,Gender_Male,Race_group_by_Alien,Race_group_by_Artificial Beings,Race_group_by_Divine and Semidivine,Race_group_by_Genetically Altered Beings,Race_group_by_Human and Variation,Race_group_by_Mythical,Race_group_by_Not Identified,...,Height,Weight,Publisher_group_by_DC Comics,Publisher_group_by_Marvel Comics,Alignment_bad,Alignment_good,Gender,Publisher_group_by,Race_group_by,Alignment
0,A-Bomb,False,True,False,False,False,False,True,False,False,...,203.0,441.0,False,True,False,True,Male,Marvel Comics,Human and Variation,good
1,Abe Sapien,False,True,False,False,False,False,False,False,False,...,191.0,65.0,False,False,False,True,Male,Others,Underwater,good
2,Abin Sur,False,True,True,False,False,False,False,False,False,...,185.0,90.0,True,False,False,True,Male,DC Comics,Alien,good
3,Abomination,False,True,False,False,False,False,True,False,False,...,203.0,441.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad
5,Absorbing Man,False,True,False,False,False,False,True,False,False,...,193.0,122.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad


In [16]:
print(pwr.shape, cod_pwr.shape)
cod_pwr.head()

(667, 168) (667, 12)


,hero_names,Physical Skills,Energy Manipulation,Elemental Powers,Mental and Psychic Powers,Space-Time Manipulation,Body Transformation,Supernatural Forces,Tech Skills,Perception Skills,Locomotion Skills,Combat Skills
0,3-D Man,3,0,0,0,0,0,0,0,0,1,0
1,A-Bomb,7,0,0,0,0,0,0,0,0,0,0
2,Abe Sapien,9,0,0,2,0,0,0,0,1,2,2
3,Abin Sur,0,0,0,0,0,0,0,1,0,0,0
4,Abomination,4,0,0,1,0,1,1,0,0,1,0


In [17]:
cod_pwr[cod_pwr.hero_names.isin(cod_inf.name)]

,hero_names,Physical Skills,Energy Manipulation,Elemental Powers,Mental and Psychic Powers,Space-Time Manipulation,Body Transformation,Supernatural Forces,Tech Skills,Perception Skills,Locomotion Skills,Combat Skills
1,A-Bomb,7,0,0,0,0,0,0,0,0,0,0
2,Abe Sapien,9,0,0,2,0,0,0,0,1,2,2
3,Abin Sur,0,0,0,0,0,0,0,1,0,0,0
4,Abomination,4,0,0,1,0,1,1,0,0,1,0
6,Absorbing Man,6,3,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
661,Yellowjacket,0,0,0,0,0,1,0,0,0,0,1
662,Yellowjacket II,0,1,0,0,0,1,0,0,0,1,0
664,Yoda,5,3,0,4,0,1,1,0,1,2,2
665,Zatanna,0,1,5,1,0,0,1,0,0,0,0


In [18]:
hero_db = pd.merge(cod_pwr.rename(columns = {'hero_names': 'name'}), cod_inf, on = 'name', how = 'inner')
hero_db.to_csv('..\\Bases de Dados\\hero_db.csv', index = False)
hero_db.head()

,name,Physical Skills,Energy Manipulation,Elemental Powers,Mental and Psychic Powers,Space-Time Manipulation,Body Transformation,Supernatural Forces,Tech Skills,Perception Skills,...,Height,Weight,Publisher_group_by_DC Comics,Publisher_group_by_Marvel Comics,Alignment_bad,Alignment_good,Gender,Publisher_group_by,Race_group_by,Alignment
0,A-Bomb,7,0,0,0,0,0,0,0,0,...,203.0,441.0,False,True,False,True,Male,Marvel Comics,Human and Variation,good
1,Abe Sapien,9,0,0,2,0,0,0,0,1,...,191.0,65.0,False,False,False,True,Male,Others,Underwater,good
2,Abin Sur,0,0,0,0,0,0,0,1,0,...,185.0,90.0,True,False,False,True,Male,DC Comics,Alien,good
3,Abomination,4,0,0,1,0,1,1,0,0,...,203.0,441.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad
4,Absorbing Man,6,3,1,0,0,1,0,0,0,...,193.0,122.0,False,True,True,False,Male,Marvel Comics,Human and Variation,bad
